# Tweets producer

We will see how to use Spark Structured Streaming with Kafka using a hashtags count example. 

First we need to install some libraries

In [ ]:
# !pip3 install requests_oauthlib
!pip3 install kafka_python
!pip3 install tweepy

-----

Now we can run the Kafka producer which is retrieving Tweets from user `@sgioldasis` and sends them to Kafka topic `tweets`

In [ ]:
import tweepy
import time
from kafka import KafkaProducer
from dateutil.parser import parse
import pytz

# Define our Twitter credentials
consumer_key='XFp79e9YHmllBxNNfmjbnEkPo'
consumer_secret='CFj6I7ejb7hYLtrj8lfUBYfxvy3OcJBx7hRDXqxKE41r0iT3md'
access_token='11824232-RlSh3OA7vEuDqm2u3cF4ZRPdlSl72IlE0VnnJRc4O'
access_token_secret='NxYfQx34vK2XUD6eZm4d1LVPylWEC0zvSZ9u9lVqMTf2J'
    
# Authorize our Twitter credentials
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

# Initialize variables
topic = "savas"
userID = "@sgioldasis"
last_processed_ts = parse('2021-11-16 07:33:00+02:00')
producer = KafkaProducer(bootstrap_servers='broker:29092')
athens = pytz.timezone('Europe/Athens')

# Loop
while True:

    # Get from Twitter API
    tweets = api.user_timeline(screen_name=userID, 
                           # 200 is the maximum allowed count
                           count=1,
                           include_rts = False,
                           # Necessary to keep full_text 
                           # otherwise only the first 140 words are extracted
                           tweet_mode = 'extended'
                           )
    
    # Get last Tweet info
    info = tweets[:1][0]
    
    # Process last Tweet
    if info.created_at > last_processed_ts:
        
        # Print Tweet (we can also print info.id)
        print(f"{info.created_at.astimezone(athens)} {info.full_text}")
        
        # Send Tweet text to Kafka
        producer.send(topic, info.full_text.encode('utf-8'))
        
        # Update last processed timestamp
        last_processed_ts = info.created_at
            
        
    # Sleep for a while
    time.sleep(10)
        
